In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
# Set up CUDA in OS
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Import libabries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchmetrics import Accuracy, Recall, Precision
from torchinfo import summary
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
from tqdm.auto import tqdm

In [3]:
from pathlib import Path
from helper import load_dotenv
from pytorch_model_tools import get_datasets, train, validate

In [4]:
dotenv = load_dotenv()

In [5]:
mlflow.set_tracking_uri(dotenv['MFLOW_DB_PATH'])
mlflow.set_experiment("moire_detection")

<Experiment: artifact_location='/Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detection/60_gebastel/Model/mlruns/1', creation_time=1709374525946, experiment_id='1', last_update_time=1709374525946, lifecycle_stage='active', name='moire_detection', tags={}>

In [6]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
available_datasets = get_datasets( dataset_directory=(dotenv['TEMP_PROCESSING_DIR'] / '..' / 'idiotentest') )

train size 200
class names ['moire', 'no_moire']


# get model

In [9]:
model = models.mobilenet_v3_small( weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1 ) 
# Add a fully-connected layer for classification
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[0].in_features, 2) #,
    #nn.Sigmoid()
)
model = model.to(device)

# Train

In [10]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
epochs = 10
loss_fn = nn.CrossEntropyLoss()
accuracy_fn = Accuracy(task="multiclass", num_classes=2).to(device)
recall_fn = Recall(task="multiclass", average='macro', num_classes=2).to(device)
precision_fn = Precision(task="multiclass", average='macro', num_classes=2).to(device)
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

with mlflow.start_run():
    params = {
        "epochs": epochs,
        "learning_rate": learning_rate,
        "batch_size": 64,
        "loss_function": loss_fn.__class__.__name__,
        "metric_function": accuracy_fn.__class__.__name__,
        "optimizer": "SGD",
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")

    for t in tqdm(range(epochs)):
        print(f"Epoch {t+1} -------------------------------", end="\n")
        train(available_datasets['train']['dataloader'], model, device, loss_fn, optimizer, metrics=[('accuracy',accuracy_fn), ('recall',recall_fn), ('precision',precision_fn)], log_intervall=1 )
        #print("validate", end="\n")
        #validate(available_datasets['val']['dataloader'], model, device, loss_fn, metrics=[('accuracy',accuracy_fn), ('recall',recall_fn), ('precision',precision_fn)] )

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 -------------------------------
[4 / 4] loss: 0.729217 accuracy: 0.488281 recall: 0.490329 precision: 0.489563
Epoch 2 -------------------------------
[4 / 4] loss: 0.670984 accuracy: 0.550781 recall: 0.569606 precision: 0.591185
Epoch 3 -------------------------------
[4 / 4] loss: 0.562550 accuracy: 0.816406 recall: 0.816565 precision: 0.851555
Epoch 4 -------------------------------
[4 / 4] loss: 0.486241 accuracy: 0.851562 recall: 0.863976 precision: 0.860799
Epoch 5 -------------------------------
[4 / 4] loss: 0.424653 accuracy: 0.898438 recall: 0.898613 precision: 0.913833
Epoch 6 -------------------------------
[4 / 4] loss: 0.392217 accuracy: 0.882812 recall: 0.873816 precision: 0.907091
Epoch 7 -------------------------------
[4 / 4] loss: 0.481324 accuracy: 0.808594 recall: 0.743304 precision: 0.817964
Epoch 8 -------------------------------
[4 / 4] loss: 0.299241 accuracy: 0.902344 recall: 0.906624 precision: 0.911831
Epoch 9 -------------------------------
[4 / 4] 